In [1]:
using ComponentArrays, Lux, DiffEqFlux, OrdinaryDiffEq, Optimization, OptimizationOptimJL,
      OptimizationOptimisers, Random, Plots
using ArrheniusModel
using RecursiveArrayTools
using Statistics
using Enzyme

In [3]:
using Dates
timestamp = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")

"2024-10-02_23-29-22"

In [7]:
# example from tests
G = [1.0,0.0]
Ea = [0. 0.2; 0.2 0.]
pe = PhaseEnergies(G, Ea)

PhaseEnergies([1.0, 0.0], [0.0 0.2; 0.2 0.0], [0.0 0.2; 1.2 0.0])

In [8]:
# try forward mode without the struct first
# this will give the derivative of each element of the output matrix wrt the 1,2 element of the input
db = Array(zero(pe.barriers))
db[1,2] = 1.0
g12 = Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db))[1]
# (convince yourself that this gives the right result)

2×2 Matrix{Float64}:
  0.0168865  -0.0168865
 -0.0         0.0

In [ ]:
db = Array(zero(pe.barriers))
db[2,2] = 1.0
g22 = Enzyme.autodiff(Forward, arrhenius_rate, Duplicated, Duplicated(pe.barriers, db))[2]


In [2]:
rng = Xoshiro(0)
G = [-5.92, -5.942, -5.97]
Ea = [0.00 1.00 0.01; 1.00 0.00 1.00; 0.01 1.00 0.00]
pe = PhaseEnergies(G, Ea)
T_range = 300.0 : 50.0 : 600.0
flow_rates = 0.5 : 0.2 : 1.5
T_grid, f_grid = meshgrid(T_range, flow_rates)
K = [arrhenius_rate(pe.barriers, T) for T in T_grid]
println("Ksize: ", size(K), " Tsize: ", size(T_grid), " fsize: ", size(f_grid))

Ksize: (6, 7) Tsize: (6, 7) fsize: (6, 7)


In [5]:
t= 10
dt = 0.05
datasize = Int(t/0.5+1)
num_steps = floor(Int, t/dt)
num_layers = floor(Int, t/0.5)+1
n = n_phases(pe)
c0 = zeros(num_layers, n, size(T_grid)...)
c0[1, 1, :, :] .= 1.0
j = 0
j0 = 0
tspan = (0.0, (num_steps-1) * dt)
tsteps = range(tspan[1], tspan[2]; length = datasize)
println("c0: ", size(c0))

c0: (21, 3, 6, 7)


In [6]:

decay_coefficients = 0.00001 * f_grid
fcoeff = flow_coefficient.("exponential", num_layers, decay_coefficients)
p = (fcoeff, j0, j, dt, num_steps, num_layers, K, T_grid)
fcoeff

6×7 Matrix{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.99994, 0.999935, 0.99993, 0.999925, 0.99992, 0.999915, 0.99991, 0.999905, 0.9999, 0.999895]       …  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.99994, 0.999935, 0.99993, 0.999925, 0.99992, 0.999915, 0.99991, 0.999905, 0.9999, 0.999895]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.999916, 0.999909, 0.999902, 0.999895, 0.999888, 0.999881, 0.999874, 0.999867, 0.99986, 0.999853]     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.999916, 0.999909, 0.999902, 0.999895, 0.999888, 0.999881, 0.999874, 0.999867, 0.99986, 0.999853]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.999892, 0.999883, 0.999874, 0.999865, 0.999856, 0.999847, 0.999838, 0.999829, 0.99982, 0.999811]     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.999892, 0.999883, 0.999874, 0.999865, 0.999856, 0.999847, 0.999838, 0.999829, 0.99982, 0.999811]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [ ]:
function deposition_rates!(dc, c, p, t)
    # Unpack parameters
    fcoeff, j0, j, dt, num_steps, num_layers, K, T_grid= p
    # Calculate deposition rates
    j = floor(Int, t / 0.5) + 1
    f = reverse.(fcoeff[j: num_layers+j-1,:,:])
    dc .= c .* f * K
    if j != j0
        c[j+1, 1] = 1.0
        j = j0
    end
end

prob = ODEProblem(deposition_rates!, c0, tspan, p)
ode_data = Array(solve(prob, Euler(), saveat = 0.5, dt = dt)) #Training data

ode_data_avg = mean(ode_data, dims=1)
ode_data_avg = reshape(ode_data_avg, (3, 21))
ode_data

In [ ]:


function ODE_calculation(barriers=pe.barriers, T=T, prob=prob)
    K = arrhenius_rate(barriers, T)
    ode_data = Array(solve(prob, Euler(), saveat = 0.5, dt = 0.05))
    return ode_data[:,:,end]
end

ODE_calculation(pe.barriers, T)

In [ ]:
db = Array(zero(pe.barriers))
dT = 1.0
godeT = Enzyme.autodiff(Forward,
 ODE_calculation,
 Duplicated(pe.barriers, db), 
 Duplicated(T,dT), 
 Const(prob))[1]
end

In [ ]:
function simple_ode!(du, u, p, t)
    a = p[1]
    du[1] = -a * u[1]
end

# Define the ODE calculation function
function ODE_calculation(u0, p, tspan)
    prob = ODEProblem(simple_ode!, u0, tspan, p)
    sol = solve(prob, Tsit5())
    return sol[end] 
end
# Define the parameters and initial conditions
u0 = [1.0]  # Initial condition
a = 2.0     # Parameter
p = [a]
tspan = (0.0, 1.0)

# Compute the sensitivity with respect to the parameter 'a' using Enzyme.autodiff
da = 1.0  # Perturbation for parameter 'a'
sensitivity = Enzyme.autodiff(Forward, ODE_calculation, Const(u0), Const(p), Const(tspan))

println("Sensitivity with respect to parameter 'a': ", sensitivity)

In [ ]:
ode_data_avg

In [ ]:
inputs = [T, flow_rate]
input_size = length(inputs)  # Replace with the actual size of `inputs` if it's not a 1D vector
Ea_size = (n ^ 2 - n) ÷ 2
fcoeff_size = length(fcoeff) #sigmoid 0~1
precoeff_size = 1
output_size = Ea_size + fcoeff_size + precoeff_size
nn = Chain(
    Dense(input_size, input_size*3*n, tanh),
    Dense(input_size*3*n, output_size*2, tanh),
    Dense(output_size*2, output_size, sigmoid)
)
#Combine activation functions and setup constraints
#Normalize the input


In [ ]:
#For debugging

dp, dst = Lux.setup(rng, nn)
model_debug = Lux.Experimental.@debug_mode nn

try
    model_debug(inputs, dp, dst)
catch e
    println(e)
end

In [ ]:
p, st = Lux.setup(rng, nn)

In [ ]:
println(j,j0,c0)

In [ ]:
function model_rates!(du, u, p, t)
    # Unpack parameters
    p_pe, p_fcoeff, p_precoeff = p
    # Calculate deposition rates
    j = floor(Int, t / 0.5) + 1
    f = reverse(p_fcoeff[j: num_layers+j-1])
    du .= p_precoeff * u .* f * p_pe.K
    if j != j0
        u[j+1, 1] = 1.0
        j = j0
    end
end

function predict_neuralode(p)
    # Get parameters from the neural network
    inputs = [0.1, flow_rate] #Normalize 300K to 0.1 at this time point
    output, outst = nn(inputs, p, st)

    # Segregate the output
    pp_Ea = output[1:Ea_size]
    p_Ea = zeros(n, n)
    index = 1
    for i in 1:n
        for j in i+1:n
            p_Ea[i, j] = pp_Ea[index]
            p_Ea[j, i] = pp_Ea[index]
            index += 1
        end
    end
    p_fcoeff = output[Ea_size+1:Ea_size+fcoeff_size]
    p_precoeff = output[end]  # The last element
    p_pe = PhaseEnergies(G, p_Ea)
    # Amorphous phase goes to zero
    nn_output = (p_pe, p_fcoeff, p_precoeff)
    arrhenius_rate(p_pe, T)

    # Define the ODE problem with the parameters from the neural network
    prob = ODEProblem(model_rates!, c0, tspan, nn_output)

    # Solve the ODE
    sol = solve(prob, Euler(), saveat = 0.5, dt = dt)
    return Array(sol)
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

T = 300.0  # Example temperature
flow_rate = 1.0  # Example flow rate
#ps = T, flow_rate, p, st, ode_data
# Predict using the NeuralODE
pred = predict_neuralode(p)
println("Training data: ", size(ode_data))
println("Prediction:", size(pred))
println("Training size: ", size(ode_data[:, :, 1]))
println(pred[:, :, 4])
println("prediction size: ", size(pred[:, :, 1]))
#!!Instability happens at 4th timestep since the weights are randomly initialized (num too large)

# Calculate loss
loss, pred = loss_neuralode(p)

println("Loss: ", loss)
println("Prediction: ", pred)

In [ ]:
println(size(ode_data[:,1,:]), ode_data[:,1,:])
println(size(ode_data))
#Average the ode_data to get the average concentration of each phase at every timestep
ode_data_avg = mean(ode_data, dims=1)
ode_data_avg = reshape(ode_data_avg, (3, 21))
println(size(ode_data_avg))
display(ode_data_avg[:,:])

In [ ]:
propertynames(nn)

In [ ]:
propertynames(nn.layers[1])

In [ ]:
# Callback function to observe training
loss_values = Float64[]
callback = function (p, l, pred; doplot = false)
    println(l)
    push!(loss_values, l)
    # plot current prediction against data
    if doplot
        pred_avg = mean(pred, dims=1)
        pred_avg = reshape(pred_avg, (3, 21))
        #plot the three phases from ode_data_avg and pred_avg
        plt = scatter(tsteps, ode_data_avg[1, :], label = "Phase 1 Data", color = :blue)
        scatter!(plt, tsteps, ode_data_avg[2, :], label = "Phase 2 Data", color = :red)
        scatter!(plt, tsteps, ode_data_avg[3, :], label = "Phase 3 Data", color = :green)
        scatter!(plt, tsteps, pred_avg[1, :], label = "Phase 1 Prediction", color = :blue, shape = :cross)
        scatter!(plt, tsteps, pred_avg[2, :], label = "Phase 2 Prediction", color = :red, shape = :cross)
        scatter!(plt, tsteps, pred_avg[3, :], label = "Phase 3 Prediction", color = :green, shape = :cross)
        display(plot(plt))
        savefig(plt, "training_$timestamp.svg")
    end
    return false
end

pinit = ComponentArray(p)

callback(pinit, loss_neuralode(pinit)...)

In [ ]:
println(typeof(pinit))

In [ ]:
# Train using the Adam optimizer
adtype = Optimization.AutoFiniteDiff()

optf = Optimization.OptimizationFunction((p,_) -> loss_neuralode(p), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(
    optprob, OptimizationOptimisers.Adam(0.02); callback = callback, maxiters = 50)


In [ ]:
plt = scatter!(loss_values, title="Loss Transformation", xlabel="Iteration", ylabel="Loss")
# Generate a unique filename using the current timestamp
timestamp = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
filename = "loss_transformation_$timestamp.svg"
# Save the plot to a file
savefig(plt, filename)
plt

In [ ]:
callback(result_neuralode.u, loss_neuralode(result_neuralode.u)...; doplot = true)
#scatter!(plt, tsteps, loss_neuralode(result_neuralode.u)[2][1, :]; label = "prediction") # hide

In [ ]:
display(predict_neuralode(result_neuralode.u)[:, :, 21])
display(ode_data[:, :, 21])

In [ ]:
inputs = [0.1, flow_rate]
trained_model = result_neuralode.u
output, outst = nn(inputs, trained_model, st)
pp_Ea = output[1:Ea_size]
p_Ea = zeros(n, n)
index = 1
for i in 1:n
    for j in i+1:n
        p_Ea[i, j] = pp_Ea[index]
        p_Ea[j, i] = pp_Ea[index]
        index += 1
    end
end
# Segregate the output
display(p_Ea)
display(pe.Ea)
p_fcoeff = output[Ea_size+1:Ea_size+fcoeff_size]
p_precoeff = output[end]
display(p_fcoeff)
display(fcoeff)
display(p_precoeff)
